# RB175 Networked Applications

## Web server:
- The web server is responsible for using the operating system’s socket API to receive information in the form of HTTP requests.

## Application server:
- an HTTP interface that can translate HTTP requests, forwarded from the web server, into sensible arguments that are then passed to the application. This interface would also translate the application’s non-HTTP response into HTTP that is then passed back to the web server and, finally, to the client.
- It will execute your application just like an object calls another object’s method in Ruby.
- It will parse HTTP information into Ruby data structures that make sense.
- It will interface with a standard web server via socket connections.
- It will capture the return value of your application and transforms the return value into HTTP-formatted text.


The application server receives HTTP text from the web server, and then the application server’s HTTP parser will format this into Ruby-friendly data structures. When the application server is ready to hand the parsed request to the Ruby app, it will call your Ruby app with a method, passing in the parsed request as an argument or series of arguments. The Ruby app will execute, and the application server will store the Ruby app’s return value in a variable. It will re-format this variable into an HTTP response that the web server will understand, and then it will send that formatted HTTP response back to the web server via a socket connection. From there, the web server sends the HTTP response to the client.


## Rack
- How do we make sure that multuple combinations of different frameworks (application) can communicate with different application servers? 
- Rack is a generic interface to help application developers connect to applications servers.
- Rack’ is basically a specification of these two things: what the server should send to the app and what the app should return to the server. 

## Sinatra

### Templating
- The ERB library can process a special syntax that mixes Ruby into HTML, and produces a final 100% HTML string.
- `<%= %>` will evaluate the embedded ruby code and include its return value in the HTML output (i.e. to call a method/variable).
- `<% %>` will only evaluate the Ruby code, but not include the return value in the HTML output (i.e. to define a method/variable).

### View helpers
- Helpers are methods that are made available in templates by Sinatra for the purpose of filtering data, processing data, or performing some other functionality.
- Helpers are defined within a helpers block in a Sinatra application.

```
helpers do
  def slugify(text)
    text.downcase.gsub(/\s+/, "-").gsub(/[^\w-]/, "")
  end
end
```

### Redirect 

#### Catch-all
```
not_found do
  redirect "/"
end
```
#### Normal redirect 
- `redirect '/path'`

### Forms
- When a form is submitted, the browser makes a HTTP request.
- This request is made to the path or URL specified in the action attribute of the form element.
- The method attribute of the form determines if the request made will use GET or POST.
- The value of any input elements in the form will be sent as parameters. The keys of these parameters will be determined by the name attribute of the corresponding input element.

### Summary
- Sinatra is a small web framework.
- HTTP requests are handled in Sinatra by creating routes for a path or set of paths.
- Routes are created using methods named after the HTTP method to be handled. So, a GET request is handled by a route defined using the get Sinatra method.
- View templates can be written in many templating languages, such as ERB. They provide a place to define the HTML display of a response outside of the route handling it. Templating languages are usually better suited to describing HTML than plain Ruby.
- A layout is a view template that surrounds a specific response's template. They are used to provide shared HTML that is used by all views, and often include links to stylesheets and JavaScript files.
- Routes can specify parameters by using colon followed by the parameter name: /chapters/:number. In this case, the value would be accessible within the route using params[:number].
- Code placed in a before block is executed before the matching route for every request.
- View helpers are Ruby methods that are used to minimize the amount of Ruby code included in a view template. These methods are defined within a helpers block in Sinatra.
- A user can be sent to a new location in response to a request with redirection. This is done in Sinatra using the redirect method.
- The redirection is accomplished by setting the Location header in the response. The client looks at the URL in the location header and sends out a new HTTP GET request for the associated resource, which in turn navigates the client to that new location.
- The files in a project can be identified as either server-side or client-side code based on where they will be evaluated.- 

### Deploying to heroku 
- Procfile defines what types of processes are provided by the application and how to start them.
  - `web: bundle exec puma -t 5:5 -p ${PORT:-3000} -e ${RACK_ENV:-development}`
- config.ru tells the web server how to start the application. In this project, we require the file that contains the Sinatra application and then start it.
```
require "./book_viewer"
run Sinatra::Application
```
- While WEBrick is a fine web server for development, it is better to use a production-ready web server such as Puma when deploying a project.
- Puma is a threaded web server, which means that it can handle more than one request at a time using a single process. As a result, Puma will perform much better for most applications.
- A specific version of Ruby can be specified in the Gemfile to ensure that the same version is used in both development and production.

## Sinatra summary
- State is data that persists over time.
- The session provides a way to store data that will persist between subsequent HTTP requests. This data is associated with a specific user by storing a cookie in their browser. In Sinatra, the session data itself is also stored in this cookie, but this is configurable and not always the case with other web frameworks.
- Data that is submitted to the server often needs to be validated to ensure it meets the requirements of the application. In this lesson we built server-side validation as we performed the validation logic on the server.
- Messages that need to be displayed to the user on their next request and then deleted can be stored in the session. This kind of message is often referred to as a flash message.
- Content from within a view template can be stored under a name and retrieved later using content_for and yield_content.
- GET requests should only request data. Any request that modifies data should be over POST or another non-GET method.
- Web browsers don't support request methods other than GET or POST in HTML forms, so there are times when a developer has to use POST even when another method would be more appropriate.
- View helpers provide a way to extract code that determines what HTML markup is generated for a view.

### Testing Sinatra applications

1. Make a request to your application.
  - Use get, post, or other HTTP-method named methods.

2. Access the response.
  - The response to your request will be accessible using last_response. 
  - This method will return an instance of Rack::MockResponse. 
  - Instances of this class provide status, body, and [] methods for accessing their status code, body, and headers, respectively.

3. Make assertions against values in the response.
  - Use the standard assertions supplied by Minitest.

```ruby
ENV["RACK_ENV"] = "test"

require "rack/test"
require 'minitest/autorun'
require "minitest/reporters"
Minitest::Reporters.use!


require_relative "../cms"

class AppTest < Minitest::Test
  include Rack::Test::Methods

  def app
    Sinatra::Application
  end

  def test_index
    get "/"
    assert_equal 200, last_response.status
    assert_equal "text/html;charset=utf-8", last_response["Content-Type"]
    assert_equal "Hello, world!", last_response.body
  end
end
```